In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

##  Data Preprocessing

In [3]:
# Loading formatted data
# I use format the data into pd dataframe
# See data_formatting.ipynb for details
train_data = pd.read_pickle("../dataset/train.pickle")
validate_data = pd.read_pickle("../dataset/validate.pickle")
test_data = pd.read_pickle("../dataset/test.pickle")

### Tokenize the source code

#### BoW

For data batching convenience, the paper trained only on functions with token length $10 \leq l \leq 500$, padded to the maximum length of **500**  
The paper does not mention to pad the 0 at the end or at the beginning, so I assume they append the padding at the end (actually, this is not a big deal in CNN)

text_to_word_sequence does not work since it ask a single string

In [ ]:
# train_tokenized = tf.keras.preprocessing.text.text_to_word_sequence(train_data[0])
# x_train = tf.keras.preprocessing.sequence.pad_sequences(train_tokenized, maxlen=500, padding="post")

In [ ]:
# validate_tokenized = tf.keras.preprocessing.text.text_to_word_sequence(validate_data[0])
# x_validate = tf.keras.preprocessing.sequence.pad_sequences(validate_tokenized, maxlen=500, padding="post")

In [ ]:
# test_tokenized = tf.keras.preprocessing.text.text_to_word_sequence(test_data[0])
# x_test = tf.keras.preprocessing.sequence.pad_sequences(test_tokenized, maxlen=500, padding="post")

#### Init the Tokenizer

#### BoW

In [20]:
# The paper does not declare the num of words to track, I am using 10000 here
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
# Required before using texts_to_sequences
# Arguments; a list of strings
tokenizer.fit_on_texts(list(train_data[0]))

For data batching convenience, the paper trained only on functions with token length $10 \leq l \leq 500$, padded to the maximum length of **500**  
The paper does not mention to pad the 0 at the end or at the beginning, so I assume they append the padding at the end (actually, this is not a big deal in CNN)

In [21]:
train_tokenized = tokenizer.texts_to_sequences(train_data[0])
x_train = tf.keras.preprocessing.sequence.pad_sequences(train_tokenized, maxlen=500, padding="post")

In [22]:
validate_tokenized = tokenizer.texts_to_sequences(validate_data[0])
x_validate = tf.keras.preprocessing.sequence.pad_sequences(validate_tokenized, maxlen=500, padding="post")

In [23]:
test_tokenized = tokenizer.texts_to_sequences(test_data[0])
x_test = tf.keras.preprocessing.sequence.pad_sequences(test_tokenized, maxlen=500, padding="post")

In [25]:
y_train = train_data[train_data.columns[1:]].astype(int)
y_validate = validate_data[validate_data.columns[1:]].astype(int)
y_test = test_data[test_data.columns[1:]].astype(int)

## Model Design

In [17]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=10000, output_dim=13, input_length=500))
#model.add(tf.keras.layers.Conv1D(filters=512, kernel_size=9, padding="same", activation="relu"))
model.add(tf.keras.layers.Conv1D(filters=512, kernel_size=9, activation="relu"))
model.add(tf.keras.layers.MaxPool1D(pool_size=3))
model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=64, activation="relu"))
model.add(tf.keras.layers.Dense(units=16, activation="relu"))
# I am using the sigmoid rather than the softmax mentioned in the paper
model.add(tf.keras.layers.Dense(units=5, activation="sigmoid"))

# Adam Optimization with the parameter stated in the paper
adam = tf.keras.optimizers.Adam(lr=0.005)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 13)           130000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 492, 512)          60416     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 164, 512)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 164, 512)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 83968)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                5374016   
_________________________________________________________________
dense_7 (Dense)              (None, 16)               

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=10, verbose=1, validation_data=(x_validate, y_validate))

Epoch 1/10
 611/7965 [=>............................] - ETA: 48:41 - loss: 0.0811 - auc_2: 0.8146